In [2]:
#prepping imports 
import pandas as pd
import rispy
import os 

ModuleNotFoundError: No module named 'pandas'

In [ ]:
#prepping systematic review data 
working_directory = os.path.abspath(os.getcwd())
campbell_df_bulk = pd.DataFrame()
cochrane_df_bulk = pd.DataFrame()
cee_df_bulk = pd.DataFrame()
for file in os.scandir(working_directory): 
    if file.name.startswith('Campbell'):
        with open (file.path, 'r') as f:
            campbell_df = pd.DataFrame(rispy.load(f))
            campbell_df_bulk = pd.concat([campbell_df_bulk, campbell_df], ignore_index=True)

    if file.name.startswith('Cochrane'): 
        with open(file.path, 'r', encoding = 'utf-8') as f: 
            cochrane_df = pd.DataFrame(rispy.load(f, encoding = 'utf-8', skip_unknown_tags=True, skip_missing_tags=True))
            cochrane_df_bulk = pd.concat([cochrane_df_bulk, cochrane_df], ignore_index=True)

    if file.name.startswith('CEE'): 
            cee_df = pd.read_csv(file,encoding='latin-1')
            cee_df_bulk = pd.concat([cee_df_bulk, cee_df], ignore_index=True)


In [ ]:
cee_min_rating = [3,4]
cee_df_bulk.rename(columns = {'3.1' : 'search_strat_rating', '3.2' : 'search_comprehensiveness'}, inplace = True)
cee_df_slice = cee_df_bulk.query('search_strat_rating.isin(@cee_min_rating) & search_strat_rating.isin (@cee_min_rating)')

In [ ]:
#

cochrane_df_sample = cochrane_df_bulk.sample(n=10, random_state=42)
campbell_df_sample = campbell_df_bulk.sample(n=10, random_state=42)
#sampling CEE data with caveat that we sample reviews that have good ratings for their search strategies, which means sampling reviews that have a rating of 3 or 4, at columns 3.1 and 3.2 
cee_min_rating_searchstrat = [4]
cee_df_bulk.rename(columns = {'3.1' : 'search_strat_rating', '3.2' : 'search_strat_ratin'}, inplace = True)
cee_df_slice = cee_df_bulk.query('search_strat_rating.isin(@cee_min_rating_searchstrat)')
cee_df_sample = cee_df_slice.sample(n=10, random_state=42)


In [ ]:
#combine all dataframes into one. All i really want is the title, abstract, and doi for each review.
desired_columns_cochrane = ['primary_title', 'abstract', 'doi']
desired_columns_campbell = ['primary_title', 'notes_abstract', 'doi']
desired_columns_cee = ['Title','Abstract', 'Link']
normalised_column_names= ['title', 'abstract', 'doi']

campbell_df_sample_slice = (campbell_df_sample[desired_columns_campbell])
campbell_df_sample_slice.columns = normalised_column_names
campbell_df_sample_slice['database_source'] = 'Campbell'

cochrane_df_sample_slice = (cochrane_df_sample[desired_columns_cochrane])
cochrane_df_sample_slice.columns = normalised_column_names
cochrane_df_sample_slice['database_source'] = 'Cochrane'

cee_df_sample_slice = (cee_df_sample[desired_columns_cee])
cee_df_sample_slice.columns = normalised_column_names
cee_df_sample_slice['database_source'] = 'CEE'

systematic_review_pool = pd.concat(
    [campbell_df_sample_slice, cochrane_df_sample_slice, cee_df_sample_slice],
    ignore_index=True)

#do some cleaning 

systematic_review_pool['doi'] = systematic_review_pool['doi'].str.replace(r'<[^<>]*>', '', regex=True)
systematic_review_pool['doi'] = systematic_review_pool['doi'].str.replace(r'.*(?<=)org/', '', regex=True)

systematic_review_pool.to_csv('systematic_review_pool.csv', index=False)


C:\Users\darrenkelvin\AppData\Local\Temp\ipykernel_19404\2587315901.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campbell_df_sample_slice['database_source'] = 'Campbell'
C:\Users\darrenkelvin\AppData\Local\Temp\ipykernel_19404\2587315901.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cochrane_df_sample_slice['database_source'] = 'Cochrane'
C:\Users\darrenkelvin\AppData\Local\Temp\ipykernel_19404\2587315901.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [ ]:
systematic_review_samples

title  \
database_source                                                         
CEE             29  Mechanisms of impact of blue spaces on human h...   
                33  A systematic review of the socio-economic impa...   
                36  How effective are strategies to control the di...   
                35  What is the effect of phasing out long-chain p...   
                39  How does roadside vegetation management affect...   
                31  The effectiveness of non-native fish removal t...   
                38  Response of benthic fauna to experimental bott...   
                30  The Different Dimensions of Livelihood Impacts...   
                27  The effectiveness of spawning habitat creation...   
                34  What are the effects of even-aged and uneven-a...   
Campbell        2   Strengthening women's empowerment and gender e...   
                6   Red light camera interventions for reducing tr...   
                9   Effects of trauma-informed approaches in schoo...   
                8   Aquaculture for improving productivity, income...   
                12  Policies and interventions to remove gender-re...   
                4   Citizen engagement in public services in low- ...   
                11  The impacts of agroforestry interventions on a...   
                3   Multiagency programs with police as a partner ...   
                0   Adult/child ratio and group size in early chil...   
                7   Interventions for improving executive function...   
Cochrane        15  Vena caval filters for the prevention of pulmo...   
                19  Atovaquone‐proguanil for treating uncomplicate...   
                22  Pharmacological interventions for pain and sed...   
                21  Selective serotonin reuptake inhibitors (SSRIs...   
                26  Pharmaceutical policies: effects of regulating...   
                17  Ab interno supraciliary microstent surgery for...   
                24  Probiotics for the prevention of Hirschsprung‐...   
                16  Pentoxifylline for the treatment of endometrio...   
                13  How effects on health equity are assessed in s...   
                20  Medical interventions for chronic rhinosinusit...   

                                                             abstract  \
database_source                                                         
CEE             29  Blue spaces have been found to have significan...   
                33  Since their widespread introduction in the 198...   
                36  Background Antibiotic resistance is a major co...   
                35  Background: There is a concern that continued ...   
                39  Background: With appropriate management, based...   
                31  In aquatic systems, biological invasions can r...   
                38  Bottom-contact fishing gears are globally the ...   
                30  Through a systematic review of peer-reviewed a...   
                27  Background: Habitat is the foundation for heal...   
                34  Background: Forest harvesting changes forest h...   
Campbell        2   Abstract Background Across the globe, gender d...   
                6   Abstract Background Road traffic crashes are a...   
                9                                                 NaN   
                8   Abstract Background A steady increase in the i...   
                12  Abstract Background Gender disparities in educ...   
                4   Background How do governance interventions tha...   
                11  Abstract Background Agroforestry, the intentio...   
                3   Abstract Background Multiagency responses to r...   
                0   Abstract Background Worldwide, a large number ...   
                7   Abstract Background The consequences for child...   
Cochrane        15  Abstract - Background Pulmonary emboli (PE), o...   
                19  Abstract - Background The Wo